In [43]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sat Jul 10 08:56:00 2017 GMT+1

@author: Evert, based on Chris' feature processing
"""

"\nCreated on Sat Jul 10 08:56:00 2017 GMT+1\n\n@author: Evert, based on Chris' feature processing\n"

In [44]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelBinarizer, Imputer
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pylab as plt

In [45]:
#Read data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(train.head())
print("\n")
print(test.head())



   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  




In [46]:
#%% preprocessing input data
imputer = Imputer(strategy = "median")
encoder = LabelBinarizer()
stdScaler = StandardScaler()

In [47]:
# refill NA data
def refillData(data, colName, **kwargs):
    if "replaceValue" in kwargs:
        data[colName].fillna(kwargs["replaceValue"], inplace = True)
    else:
        data[colName].fillna(data[colName].mean(), inplace = True)

In [48]:
# label categorical data
def binaryEncoding(data, colName, encoder):
    data[colName] = encoder.fit_transform(data[colName])

#refillData(train, "Age", replaceValue = 0)
#refillData(train, "Embarked", replaceValue = "unknown")

binaryEncoding(train, "Sex", encoder)
binaryEncoding(test, "Sex", encoder)

In [49]:
# drop non-numeric columns
columns2drop = ["PassengerId", "Survived", "Name", "Ticket", "Cabin", "SibSp", "Parch", "Embarked", "Fare" ]
train_num = train.drop(columns2drop, axis = 1)
columns2drop = ["PassengerId", "Name", "Ticket", "Cabin", "SibSp", "Parch", "Embarked", "Fare" ]
test_num = test.drop(columns2drop, axis = 1)

In [50]:
# refill the missing numeric data with median
imputer.fit(train_num)
X = imputer.transform(train_num)
X_test = imputer.transform(test_num)

In [51]:
# Scale age
# print(X)

from sklearn.preprocessing import scale

X[:,2] = scale(X[:,2])
X_test[:,2]= scale(X_test[:,2])

# print(X)

In [52]:
## encode the categorical data into one-hot data
#embark = encoder.fit_transform(train["Embarked"])

# concatenate the data together
#X = np.c_[X, embark]
y = train["Survived"]
#X[:,-1] = stdScaler.fit_transform(X[:, -1])

In [53]:
# Select optimal hyperparameters for support vector 
#
#
# WARNING: May take some time to complete
# Consider cutting back on the number of cross-validations, 
# currently 3, by specifying 'cv = 1' or 'cv = 2' in the GridSearchCv call.

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf', 'poly'), 'C':[.001, .01, 100]}
svClf = SVC(C = 1, kernel = 'rbf', probability = True)
clf = GridSearchCV(svClf, parameters)
clf.fit(X, y)
print(clf.score(X, y))
print(clf.cv_results_)
print(clf.best_score_)

# Note that optimal model is automatically fitted to clf after this command. 

# Or, short-cutting the procedure:
# clf = SVC(C = 100, kernel = 'rbf', probability = True)

0.812570145903
{'std_train_score': array([ 0.        ,  0.        ,  0.02666578,  0.0103778 ,  0.        ,
        0.00396805,  0.00619829,  0.0079361 ,  0.00901368]), 'rank_test_score': array([7, 7, 6, 4, 7, 5, 2, 1, 3]), 'mean_score_time': array([ 0.00300002,  0.00866667,  0.00666666,  0.00433334,  0.00833321,
        0.00400003,  0.00233324,  0.00533334,  0.00300002]), 'std_test_score': array([ 0.        ,  0.        ,  0.06472669,  0.04928058,  0.        ,
        0.03089989,  0.01239659,  0.01802736,  0.02598392]), 'split1_train_score': array([ 0.61616162,  0.61616162,  0.75252525,  0.76936027,  0.61616162,
        0.73232323,  0.77946128,  0.80976431,  0.80808081]), 'split0_test_score': array([ 0.61616162,  0.61616162,  0.60942761,  0.69360269,  0.61616162,
        0.68350168,  0.78787879,  0.76767677,  0.75757576]), 'mean_test_score': array([ 0.61616162,  0.61616162,  0.69135802,  0.76318743,  0.61616162,
        0.72502806,  0.78675645,  0.79124579,  0.78226712]), 'split2_train

In [ ]:
# Plot validation curve
#
# Validation curve shows the model's predictive utility at various levels of the chosen parameter
# Here, parameter is coefficient penalty C

import matplotlib.pyplot as plt
import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import validation_curve

print(clf.get_params().keys())

train_scores, test_scores = validation_curve(
    clf, X, y, param_name="estimator__C", param_range=[0.1, 10, 100],
    cv=3, scoring="accuracy", n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with SVM")
plt.xlabel("$\Penalty \(C\)")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")

['n_jobs', 'verbose', 'estimator__gamma', 'estimator__decision_function_shape', 'estimator__probability', 'param_grid', 'cv', 'scoring', 'estimator__cache_size', 'estimator__verbose', 'pre_dispatch', 'estimator__kernel', 'fit_params', 'estimator__max_iter', 'refit', 'iid', 'estimator__shrinking', 'estimator__degree', 'estimator__class_weight', 'estimator__C', 'estimator__random_state', 'return_train_score', 'estimator', 'estimator__coef0', 'error_score', 'estimator__tol']


In [60]:
#Show confusion matrix
clf.fit(X, y)
print(clf.score(X, y))

prediction = clf.predict(X)
compare = pd.concat([pd.Series(y), pd.Series(prediction, name = 'predicted')], axis = 1)
print(compare)

compare['tp'] = 0
compare.loc[((compare['Survived']==1)&(compare['predicted']==1)), 'tp']=1
compare['tn'] = 0
compare.loc[((compare['Survived']==0)&(compare['predicted']==0)), 'tn']=1
compare['fp'] = 0
compare.loc[((compare['Survived']==0)&(compare['predicted']==1)), 'fp']=1
compare['fn'] = 0
compare.loc[((compare['Survived']==1)&(compare['predicted']==0)), 'fn']=1

print("True positives: ", compare['tp'].sum(), "\n")
print("True negatives: ", compare['tn'].sum(), "\n")
print("False positives: ", compare['fp'].sum(), "\n")
print("False negatives: ", compare['fn'].sum(), "\n")

# We can crossref the 'False' label scores with potential features that may increase predictive utility
# Patterns will tell us something of the potential for additional predictive validity of these features.


0.812570145903
     Survived  predicted
0           0          0
1           1          1
2           1          1
3           1          1
4           0          0
5           0          0
6           0          0
7           0          0
8           1          1
9           1          1
10          1          0
11          1          1
12          0          0
13          0          0
14          0          0
15          1          1
16          0          0
17          1          0
18          0          1
19          1          1
20          0          0
21          1          0
22          1          0
23          1          0
24          0          0
25          1          0
26          0          0
27          0          0
28          1          1
29          0          0
..        ...        ...
861         0          0
862         1          1
863         0          1
864         0          0
865         1          1
866         1          1
867         0          0
868       

In [ ]:
# plot ROC curve and area under curve
#
# ROC curve visualizes the predictive utility of our model

from sklearn.metrics import roc_curve, auc

y_pred = clf.predict(X)
fpr, tpr, thresholds = roc_curve(y, y_pred, pos_label=1)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#%% classifier testing

rforestClf = RandomForestClassifier(n_estimators = 5, max_features = 3, bootstrap = False)
rforestClf.fit(X, y)
cross_val_score(rforestClf, X, y, cv = 3, scoring= "accuracy")

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

train_sizes, train_scores, test_scores = learning_curve(
        rforestClf, X, y, cv=cv, n_jobs=4, train_sizes=np.linspace(.1, 1.0, 5))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

print "test score: {0:.2f}%, training score {1:.2f}%".format(test_scores_mean.mean()*100, train_scores_mean.mean()*100)


In [ ]:
#%% classifier testing

rforestClf = RandomForestClassifier(n_estimators = 5, max_features = 3, bootstrap = False)
rforestClf.fit(X2, y)
cross_val_score(rforestClf, X2, y, cv = 3, scoring= "accuracy")

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

train_sizes, train_scores, test_scores = learning_curve(
        rforestClf, X2, y, cv=cv, n_jobs=4, train_sizes=np.linspace(.1, 1.0, 5))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

print "test score: {0:.2f}%, training score {1:.2f}%".format(test_scores_mean.mean()*100, train_scores_mean.mean()*100)

test_scores_mean2 = np.mean(test_scores, axis = 0)
y2 = y
fpr2, tpr2, thresholds1 = roc_curve(y2, test_scores_mean2, pos_label=1)

In [ ]:
#%% classifier testing

rforestClf = RandomForestClassifier(n_estimators = 5, max_features = 3, bootstrap = False)
rforestClf.fit(X3, y)
cross_val_score(rforestClf, X3, y, cv = 3, scoring= "accuracy")

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

train_sizes, train_scores, test_scores = learning_curve(
        rforestClf, X3, y, cv=cv, n_jobs=4, train_sizes=np.linspace(.1, 1.0, 5))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

print "test score: {0:.2f}%, training score {1:.2f}%".format(test_scores_mean.mean()*100, train_scores_mean.mean()*100)

test_scores_mean3 = np.mean(test_scores, axis = 0)
y3 = y
fpr3, tpr3, thresholds3 = roc_curve(y3, test_scores_mean3, pos_label=1)

In [ ]:
plt.figure()
plt.title("random forest classifier")
plt.xlabel("Training examples")
plt.ylabel("Score")

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")


In [ ]:

#%%
f, ax = plt.subplots()
n_features = X.shape[1]
width = 0.5
ax.bar(range(n_features), rforestClf.feature_importances_, width)
ax.grid()
ax.set_xticks(range(n_features))
ax.set_xticklabels(train_num.columns)
ax.set_ylabel(' Feature Importance')
f.show()


In [ ]:

#%% test score for test sets
#
#binaryEncoding(test, "Sex", encoder)
#
## drop non-numeric columns
#test_num = test.drop(columns2drop, axis = 1)
#
## refill the  missing numeric data with median
#imputer.fit(test_num)
#Xtest = imputer.transform(test_num)
#
#ytest = test["Survived"]
#
#y_pred = rforestClf.predict(Xtest)
#test_score = accuracy_score(ytest, y_pred)
#
#print "test_score = {:.3f}%".format(test_score*100)